In [ ]:
pip install ipywidgets

In [1]:
from qdrant_client import QdrantClient, models

C:\projects\ask_my_resume\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient("http://localhost:6333")

In [3]:
import os
print(os.getcwd())

C:\projects\ask_my_resume


In [4]:
import json

with open("../data/resume.json", "r", encoding="utf-8") as f:
    document_raw = json.load(f)

In [5]:
document_raw

[{'id': 'ae0142cf-58e4-408a-b2c3-950b9864f8e9',
  'text': 'Work Experience\nSecuriti, Karachi, Pakistan\t11/2024 – Current\nData Scientist',
  'metadata': {'section': 'Work Experience'}},
 {'id': '81b009b4-ca83-4f13-8549-684bceb6d767',
  'text': 'Developed AI and Machine Learning solutions to process sensitive data securely, managing the full ML lifecycle from data collection and preparation to model training, evaluation, and production-readiness.',
  'metadata': {'section': 'Work Experience'}},
 {'id': 'e57bac5c-03ec-41a4-9335-9ed4737640a7',
  'text': 'Built a CNN-based model to detect the header row location within semi-structured tabular files leveraging sentence embeddings and cosine similarity, achieving 96% accuracy, <250ms inference latency, and a broader context window compared to the previous system.',
  'metadata': {'section': 'Work Experience'}},
 {'id': '3e1e2fd4-45c5-47dd-b4e6-f68835f53f6d',
  'text': 'Integrated PyTorch models into a Java-based production pipeline for per

In [6]:
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [7]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [8]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [ ]:
#to delete a collection
# client.delete_collection(collection_name=collection_name)

In [9]:
# Define the collection name
collection_name = "resume-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

ResponseHandlingException: timed out

In [11]:
points = []

for i, doc in enumerate(document_raw):
    point = models.PointStruct(
        id=doc["id"],  # or use id=i
        vector=models.Document(text=doc["text"], model=model_handle),
        payload={
            "text": doc["text"],
            "section": doc["metadata"]["section"]
        }
    )
    points.append(point)

In [12]:
client.upsert(
    collection_name="resume-rag",
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
C:\projects\ask_my_resume\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nida Khan\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run 

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [33]:
print(search("Is there any experience with java?").points[0].payload['text'])

Skills
Programming: Python, R, Java, SQL, Bash
Data Engineering & MLOps: Spark, Docker, MLflow, Git, GCP
Machine Learning & Modeling: Probability & Statistics, Deep Learning, LLM fine-tuning, Feature Engineering, Model Evaluation, Data Visualization (ggplot, Streamlit, Tableau)
Languages: English (C2), Urdu (mother tongue)


In [31]:
client.create_payload_index(
    collection_name="resume-rag",
    field_name="section",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
def search_in_experience(query, section="Work Experience", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="section",
                    match=models.MatchValue(value=section)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [39]:
print(search_in_experience("Is there any experience with java?", "Work Experience").points[0].payload['text'])

Integrated PyTorch models into a Java-based production pipeline for performance testing, collaborating closely with engineering teams to ensure seamless deployment.


In [ ]:
def vector_search(question):
    print('vector_search is used')

    query_points = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="section",
                    match=models.MatchValue(value="Work Experience")
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    # Extract just the payloads (text + section)
    results = []
    for point in query_points.points:
        results.append(point.payload)
    
    return results


In [42]:
from openai import OpenAI

openai_client = OpenAI()

ModuleNotFoundError: No module named 'openai'

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You are a resume assistant. Use the CONTEXT below to answer the QUESTION based only on the candidate's resume.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}

If the answer is not found in the context, reply with "Not available in the resume.
""".strip()

    context = ""
    # for doc in search_results:
    #     context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    for doc in search_results:
        section = doc.payload.get("section", "Unknown Section")
        text = doc.payload.get("text", "")
        context += f"[{section}] {text}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
